# LIVERPOOL v. FULHAM - match analysis
*This notebook intends to walk through the theory and calculations I am currently using in my model to calculate probability of 1 x 2 result and a table of over / under goal projections. I will break the calculations out into utility functions stored in `../src/utils` for ease of use in future explorations. This is to document the calculations.*

### Instructions:

1. Ensure all cells in all notebooks in `/notebooks` directory have been run.
2. Define `home_side` and `away_side` in first cell
3. Click 'run all'
4. Probability for 1 x 2 and over / under can be viewed in the last cell
5. *Suggestion:* copy and paste notebook to `../output/< matchday_directory >/` and rename file. IMPORTANT: Update file path in lines 14 and 15 if needed if you make a copy.



### Import League Data & Define Sides

Define 'home_side' and 'away_side'
Ensure correct output



In [32]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

"""
BUG: if copying from `notebooks/03_analysis_template_notebook.ipynb` to a directory in output
for storage, ensure the file path for reading the csv will need to be updated from '../data/processed/home_table.csv'
to '../../data/processed/home_table.csv' if frames are run again.

TLDR
if you are in ./notebooks/03_analysis_tempalte.csv you may run this cell as is. If you are not, check that following
read_csv commands read from the correct directory.
"""
home_dataframe = pd.read_csv('../data/processed/home_table.csv')
away_dataframe = pd.read_csv('../data/processed/away_table.csv')

"""

!!! INPUT TEAMS HERE !!!

wrapped in quotes, eg.

home_side = "Arsenal"
away_side = "Everton"

If there is a ValueError,
(1) check spelling against the error message. Must be exact.
(2) check capitalization
(3) check that it is wrapped in quotes

"""
home_side = "Liverpool"
away_side = "Fulham"
matchday = "2024-12-14"

# validate input
clubs = set(home_dataframe['Team'].values)
if home_side not in clubs:
    raise ValueError(f"Invalid entry for 'home_side'. Expected one of {clubs}, but received '{home_side}'")
elif away_side not in clubs:
    raise ValueError(f"Invalid entry for 'away_side'. Expected one of {clubs}, but received '{away_side}'")
else:
    print(f"Ready to calculate probability for {home_side} v. {away_side}")

Ready to calculate probability for Liverpool v. Fulham


### Calculate Projected Goals

In [33]:
# 2. grab their gpm_scored and gpm_conceded
home_side_gpm_scored = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_scored"
].values[0]

home_side_gpm_conceded = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_conceded"
].values[0]

print(home_side_gpm_scored, home_side_gpm_conceded)

away_side_gpm_scored = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_scored"
].values[0]

away_side_gpm_conceded = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_conceded"
].values[0]

print(away_side_gpm_scored, away_side_gpm_conceded)

# 3.
epl_home_avg_gpm_scored = round(sum(home_dataframe["gpm_scored"].values) / 20, 2)
epl_home_avg_gpm_conceded = round(sum(home_dataframe["gpm_conceded"].values) / 20, 2)
print(epl_home_avg_gpm_scored, epl_home_avg_gpm_conceded)

# 4. create a ratio (attack and defense rating) of their gpm to the league average gpm
home_attack_rating = home_side_gpm_scored / epl_home_avg_gpm_scored
away_defense_rating = away_side_gpm_conceded / epl_home_avg_gpm_scored
away_attack_rating = away_side_gpm_scored / epl_home_avg_gpm_conceded
home_defense_rating = home_side_gpm_conceded / epl_home_avg_gpm_conceded

print(home_attack_rating, away_defense_rating, away_attack_rating, home_defense_rating)

# home projected goals
home_projected_goals = home_attack_rating * away_defense_rating * epl_home_avg_gpm_scored
# away projected goals
away_projected_goals = away_attack_rating * home_defense_rating * epl_home_avg_gpm_conceded
print("\n ================== projected_goals ==================== \n")
print(f"HOME {home_side} projected goals:", home_projected_goals)
print(f"AWAY {away_side} projected goals: ", away_projected_goals)

1.8571428571428568 0.4285714285714285
1.1428571428571428 1.0
1.63 1.38
1.1393514460999121 0.6134969325153374 0.8281573498964804 0.3105590062111801

 ================== projected_goals ==================== 

HOME Liverpool projected goals: 1.1393514460999121
AWAY Fulham projected goals:  0.35492457852706294


### Poisson Distribution of Goal Probabilities

In [34]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)
print(f"{home_side} v. {away_side}", exact_score_prob)

Liverpool v. Fulham [[2.24411014e-01 2.55683013e-01 1.45656405e-01 5.53179454e-02
  1.57566453e-02 3.59047132e-03 6.81801448e-04 1.10973066e-04
  1.58046655e-05]
 [7.96489846e-02 9.07481857e-02 5.16970383e-02 1.96336985e-02
  5.59242068e-03 1.27434652e-03 2.41988091e-04 3.93870688e-05
  5.60946423e-06]
 [1.41346911e-02 1.61043808e-02 9.17427477e-03 3.48424107e-03
  9.92443777e-04 2.26148450e-04 4.29437607e-05 6.98971940e-06
  9.95468364e-07]
 [1.67224976e-03 1.90528019e-03 1.08539187e-03 4.12214265e-04
  1.17414230e-04 2.67552145e-05 5.08059872e-06 8.26941071e-07
  1.17772063e-07]
 [1.48380636e-04 1.69057692e-04 9.63080629e-05 3.65762436e-05
  1.04182990e-05 2.37402081e-06 4.50807340e-07 7.33754278e-08
  1.04500500e-08]
 [1.05327869e-05 1.20005460e-05 6.83641972e-06 2.59636157e-06
  7.39542076e-07 1.68519667e-07 3.20005210e-08 5.20854856e-09
  7.41795916e-10]
 [6.23057493e-07 7.09881456e-07 4.04402232e-07 1.53585422e-07
  4.37469433e-08 9.96862862e-09 1.89296191e-09 3.08106984e-10
  4.

### Under Market

In [35]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) <= t
    # print(mask)
    sums.append(exact_score_prob[mask].sum())

# print(sums)
under_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})
under_market_df["Implied Odds"] = 1 / under_market_df["Prob"]
print("~  ~ under market ~  ~\n", f"{home_side} v. {away_side}\n", under_market_df)

~  ~ under market ~  ~
 Liverpool v. Fulham
    Goals      Prob  Implied Odds
0    0.5  0.224411      4.456109
1    1.5  0.559743      1.786534
2    2.5  0.810282      1.234138
3    3.5  0.935074      1.069434


In [36]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) >= t
    sums.append(exact_score_prob[mask].sum())

# print(sums)
over_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})

over_market_df["Implied Odds"] = 1 / over_market_df["Prob"]
print("~  ~ over market ~  ~\n", f"{home_side} v. {away_side}\n", over_market_df)

~  ~ over market ~  ~
 Liverpool v. Fulham
    Goals      Prob  Implied Odds
0    0.5  0.775586      1.289348
1    1.5  0.440254      2.271417
2    2.5  0.189714      5.271079
3    3.5  0.064923     15.402891


### 1 x 2 Probability

In [37]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)

# print(exact_score_prob)
home_win_prob = np.sum(np.triu(exact_score_prob, k=1))
away_win_prob = np.sum(np.tril(exact_score_prob, k=-1))
draw_prob = np.trace(exact_score_prob)

# prepare data in readable dataframe
data = {
    "winner": [f"{home_side} (1)", "Draw (x)", f"{away_side} (2)"],
    "prob": [home_win_prob, draw_prob, away_win_prob],
}
result_df = pd.DataFrame(data)
result_df['implied_odds'] = 1 / result_df['prob']
print("      ~  ~ moneyline market ~  ~\n", result_df)

      ~  ~ moneyline market ~  ~
           winner      prob  implied_odds
0  Liverpool (1)  0.560204      1.785063
1       Draw (x)  0.324756      3.079232
2     Fulham (2)  0.115036      8.692927


# Full Results Summary:

In [38]:
# helper function to translate discrepency between Understat and Odds-Api naming
def translate(string):
    if string == "Ipswich":
        return "Ipswich Town"
    else:
        return string

home_side, away_side = translate(home_side), translate(away_side)

match_string = f"{home_side} v. {away_side}"

print(f"CALCULATED PROBABILITIES FOR {home_side} v. {away_side}")
print("\n\n        ~ ~ moneyline market ~ ~\n\n", result_df)
print("\n\n          ~ ~ over market ~ ~ \n\n", over_market_df)
print("\n\n          ~ ~ under market ~ ~ \n\n", under_market_df)

odds_df = pd.read_csv(f"../data/processed/odds/odds_matchday_{matchday}.csv")
filtered_odds = odds_df.loc[odds_df["match"] == match_string]
implied = {
    "1": 1 / filtered_odds["1"],
    "x": 1 / filtered_odds["x"],
    "2": 1 / filtered_odds["2"],
    "point": filtered_odds["ovr_und_point"],
    "over": 1 / filtered_odds["over"],
    "under": 1 / filtered_odds["under"]
}
implied_odds_df = pd.DataFrame(implied)

print(f"\n ~~ MyBookie Odds for {match_string} ~~\n",filtered_odds.drop(columns=["match"]))
print(f"\n ~~ MyBookie Odds IMPLIED for {match_string} ~~\n", implied_odds_df)

CALCULATED PROBABILITIES FOR Liverpool v. Fulham


        ~ ~ moneyline market ~ ~

           winner      prob  implied_odds
0  Liverpool (1)  0.560204      1.785063
1       Draw (x)  0.324756      3.079232
2     Fulham (2)  0.115036      8.692927


          ~ ~ over market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.775586      1.289348
1    1.5  0.440254      2.271417
2    2.5  0.189714      5.271079
3    3.5  0.064923     15.402891


          ~ ~ under market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.224411      4.456109
1    1.5  0.559743      1.786534
2    2.5  0.810282      1.234138
3    3.5  0.935074      1.069434

 ~~ MyBookie Odds for Liverpool v. Fulham ~~
       1    x    2  ovr_und_point  over  under
1  1.31  5.4  7.9            3.5  2.16   1.65

 ~~ MyBookie Odds IMPLIED for Liverpool v. Fulham ~~
           1         x         2  point      over     under
1  0.763359  0.185185  0.126582    3.5  0.462963  0.606061


# Conclusion and Pick

This match is screaming no bet. Very little value in a Liverpool win with MyBookie valuing the result at 76%. Our model gives a 6% chance of over 3.5 goals and a 93% chance of under. The caluculation for under feels to good to be true especially with the form that liverpool is in. I don't like the odds.

💡 I am curious what the model would look like if we base on the last 6 games to see the recent trend of a club. That, we'll save for another iteration of the model

### My Pick:

**NO BET**